# Imports and connection

In [5]:
import math
import numpy as np
import pandas as pd
from scipy.stats import norm

import datetime as dt
import time
import logging

import black_scholes
import libs

from optibook.synchronous_client import Exchange
import functions as fun

In [27]:
exchange = Exchange()
exchange.connect()

logging.getLogger('client').setLevel('ERROR')

2022-04-01 08:13:34,669 [asyncio   ] [MainThread  ] Using selector: EpollSelector


# Algorithm

In [13]:
position_limit = 100

# Obtain pair objects which hold all relevant information for each of the three tradeable pairs
(pair_adyen_kpn, pair_adyen_tkwy, pair_kpn_tkwy) = fun.get_tradeable_pairs()

# Main loop
while True:
    
    # During each main loop, run the procedure for each of the three tradeable pairs
    for pair in [pair_adyen_kpn, pair_adyen_tkwy, pair_kpn_tkwy]:
        instrument_a = pair.instrument_a
        instrument_b = pair.instrument_b
        constant = pair.constant
        slope = pair.gamma
        # Hedge Ratio: gamma * y / x. Use a pre-calculated ratio based on historical data 
        hedge_ratio = pair.hedge_ratio
        # Threshold: the value of Z for which (times it occurs * profit when it occurs) is maximal - pair specific
        threshold = pair.optimal_threshold
        
        # Obtain relevant trading information
        position_a, position_b, order_book_a, order_book_b = fun.get_positions_and_order_book(exchange, instrument_a, instrument_b)
        if fun.order_books_are_not_empty(order_book_a, order_book_b):
            
            # Extract information from the order books, and use this to calculate two Z-values
            # Invariant: Z1 < Z2
            best_ask_a, best_bid_a = order_book_a.asks[0], order_book_a.bids[0]
            best_ask_b, best_bid_b = order_book_b.asks[0], order_book_b.bids[0]
            z_1 = - constant - slope * np.log(best_ask_b.price) + np.log(best_bid_a.price)
            z_2 = - constant - slope * np.log(best_bid_b.price) + np.log(best_ask_a.price)
            
            # Case where A is overvalued compared to B; short A and go long in B
            if(z_1 > threshold):
                # In the pair, B is always the (in absolute terms) cheaper stock. Therefore, we always have more absolute position in B
                # when properly hedged (which is maintained as an invariant of the main loop termination), and thus we only need to check B's limits.
                if(position_b < position_limit):
                    order_size = min(position_limit - position_b, best_ask_b.volume)
                    exchange.insert_order(instrument_b, price=best_ask_b.price, volume=order_size, side='bid', order_type='ioc')
                    # Verify whether and how much of the order on B came through
                    # If it (partly) came through, hedge in A according to the hedging ratio
                    fulfilled_volume = abs(exchange.get_positions()[instrument_b] - position_b)
                    if(fulfilled_volume):
                        exchange.insert_order(instrument_a, price=best_bid_a.price, volume=int(round(fulfilled_volume/hedge_ratio, 0)), side='ask', order_type='ioc')
                        
            # Case where B is overvalued compared to A; short B and go long in A
            # Since Z1 < Z2 always holds, at most 1 of these if-statements can fire per iteration
            # The following is otherwise simply mirroring the previous block of code
            if(z_2 < -threshold):
                if(position_b > -position_limit):
                    order_size = min(position_limit + position_b, best_bid_b.volume)
                    exchange.insert_order(instrument_b, price=best_bid_b.price, volume=order_size, side='ask', order_type='ioc')
                    fulfilled_volume = abs(exchange.get_positions()[instrument_b] - position_b)
                    if(fulfilled_volume):
                        exchange.insert_order(instrument_a, price=best_ask_a.price, volume=int(round(fulfilled_volume/hedge_ratio, 0)), side='bid', order_type='ioc')
                        
        # Sometimes, the second order does not fall through, thus, we check whether we are properly hedged. 
        # If not, immediately hedge at the best available costs. 
        # Thus, we maintain the invariant that we are always properly hedged during termination of the main loop.
        position_a, position_b, order_book_a, order_book_b = fun.get_positions_and_order_book(exchange, instrument_a, instrument_b)
        while fun.unhedged(position_a, position_b, hedge_ratio):
            if fun.order_books_are_not_empty(order_book_a, order_book_b):
                fun.try_hedge(exchange, position_a, position_b, order_book_a, instrument_a, hedge_ratio)
            position_a, position_b, order_book_a, order_book_b= fun.get_positions_and_order_book(exchange, instrument_a, instrument_b)
                

IndentationError: expected an indented block (<ipython-input-42-0147fce04ed7>, line 65)

2022-04-01 08:53:03,006 [client    ] [Thread-8    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.
